# OpenAI DEMO

使用 API2D 代理

In [1]:
!pip install langchain openai

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


## 封装 OpenAI 代理

In [6]:
from langchain import PromptTemplate, OpenAI, LLMChain
import llm_config # 内容参考 llm_config.example.py

llm=OpenAI(
    temperature=0,
    openai_api_key=llm_config.MY_OPENAI_API_KEY,
    openai_api_base=llm_config.MY_OPENAI_API_BASE
)

## Chain 示例

In [7]:
# 简单场景
template = """
你是一个智能助理，要求从用户输入信息中找到用户的真实诉求。
如用户输入信息为“买杯可乐”，你的回答应该是“用户需要购买一杯可乐”。
要求有2个：
1、不允许包含任何之外的冗余信息。
2、只允许判断用户的购买需求，对其他需求直接回答“不知道”。
现在用户的输入信息是: {userInput}
"""
prompt = PromptTemplate(template=template, input_variables=["userInput"])
# print(prompt)
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.run("帮我买杯咖啡")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Invalid response object from API: '{"object":"error","message":"The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations","code":50098}' (HTTP response code was 404).


KeyboardInterrupt: 

In [51]:
# 复杂场景
menu = """
咖啡，20元，可定制冰镇、常温、热三种温度，提神解乏
白开水，3元，可加冰块，健康快速补水
可乐，5元，碳酸饮料，激爽开心
大力杯，10元，运动补水补盐，内涵左旋促进锻炼效果
"""

template = """
你是一个智能助理，会结合菜单，针对用户的咨询提供饮品购买建议，并回复购买商品详细信息。
如用户咨询为“买杯可乐”，你的回答应该是“用户需要购买一杯可乐”。
要求有2个：
1、不允许包含任何之外的冗余信息。
2、只允许判断用户的购买需求，对其他需求直接回答“不知道”。
你的菜单如下
```
{menu}
```
现在用户的咨询是 {userInput}
"""
prompt = PromptTemplate(template=template, input_variables=["userInput", "menu"])
# print(prompt)
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.run({
    "userInput": "推荐一款健身时候喝的饮料",
    "menu": menu
})

'\n你的回答应该是“推荐大力杯，10元，运动补水补盐，内涵左旋促进锻炼效果”。'

## Memory 记忆上下文

In [59]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


memory=ConversationBufferMemory()
# 补充设定记忆
memory.save_context(
    {"input":"你是谁"}, 
    {"output":"""
        一个智能助理，会结合菜单，针对用户的咨询提供饮品购买建议，并回复购买商品详细信息和价格
        如用户咨询为“买杯可乐”，你的回答应该是“用户需要购买一杯可乐，价格5元”。
        要求有2个：
        1、不允许包含任何之外的冗余信息。
        2、只允许判断用户的购买需求，对其他需求直接回答“不知道”。
        """
    }
)
# 补充菜单记忆
memory.save_context(
    {"input":"你们有哪些饮料？"},
    {"output": menu}
)

conversation = ConversationChain(
    llm=llm,
    memory=memory
)

conversation.run("推荐一款解渴的饮料")

' 我推荐白开水，3元，可加冰块，健康快速补水。'

In [60]:
conversation.run("来一杯可乐和一杯咖啡")

' 用户需要购买一杯可乐，价格5元，和一杯咖啡，价格20元。'

## Agent 和 Tool

In [63]:
# 最简单使用场景
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import BaseTool, StructuredTool, Tool, tool


tools = load_tools([
    "llm-math"
], llm=llm)
tools.append(Tool.from_function(
    func=conversation.run,
    name="SmartWaiterForDrinks",
    description="an intelligent assistant, tasked with identifying the user's true demand from their input information.",
))
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("来一杯可乐和一杯咖啡")



> Entering new AgentExecutor chain...
 I need to figure out what the user wants
Action: SmartWaiterForDrinks
Action Input: User wants one Coke and one coffee
Observation:  
User needs to purchase one Coke, priced at 5 dollars, and one coffee, priced at 20 dollars.
Thought: I now know the final answer
Final Answer: The user needs to purchase one Coke and one coffee, for a total of 25 dollars.

> Finished chain.


'The user needs to purchase one Coke and one coffee, for a total of 25 dollars.'

### 完整的复杂场景

In [83]:
# 需求识别
requirementChain = LLMChain(
    prompt=PromptTemplate(template="""
你是一个服务员，会结合菜单，针对用户的咨询提供饮品购买建议，并回复格式化的商品详细信息。
如用户咨询为“买杯可乐”，你的回答应该是"可乐,1”。
要求有2个：
1、不允许包含任何之外的冗余信息，如“回答：”。
2、只允许判断用户的购买需求，对其他需求直接回答“不知道”。
你的菜单如下
```
咖啡，20元，可定制冰镇、常温、热三种温度，提神解乏
白开水，3元，可加冰块，健康快速补水
可乐，5元，碳酸饮料，激爽开心
大力杯，10元，运动补水补盐，内涵左旋促进锻炼效果
```
现在用户的输入信息是: {userInput}
""", input_variables=["userInput"]),
    llm=llm)
requirementChain.run("推荐一款健身时候喝的饮料")

'\n回答：大力杯，10元，运动补水补盐，内涵左旋促进锻炼效果'

In [77]:
menuChain = LLMChain(
    prompt=PromptTemplate(
        template="""
你是一个餐厅会计，会针对客户购买信息给出对应商品价格。
商品菜单如下
```
咖啡，20元，可定制冰镇、常温、热三种温度，提神解乏
白开水，3元，可加冰块，健康快速补水
可乐，5元，碳酸饮料，激爽开心
大力杯，10元，运动补水补盐，内涵左旋促进锻炼效果
```
客户购买信息为：{purchaseInfo}
""",
        input_variables=["purchaseInfo"]
    ),
    llm=llm
)
menuChain.run("回答：大力杯，10元，运动补水补盐，内涵左旋促进锻炼效果")

'\n答：客户购买的商品是大力杯，价格为10元。'

In [95]:
tools = load_tools([
    "llm-math","human"
], llm=llm)

tools.append(Tool.from_function(
    func=conversation.run,
    name="watress",
    description="an intelligent assistant, tasked with identifying the user's true demand from their input information and tell the total price of them."
))

tools.append(Tool.from_function(
    func=menuChain.run,
    name="menuChecker",
    description="Check what are bought and help custom to purchase"
))

def purchaseFun(creditID: str, password: str, money: str) -> str:
    """Let human input credit card id and password to pay the money"""
    return "success" # always

tools.append(StructuredTool.from_function(purchaseFun))

agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("来杯可乐，再加二杯咖啡")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "menuChecker",
  "action_input": "来杯可乐，再加二杯咖啡"
}
```


Observation: 
根据客户购买信息，商品总价为：5元+（20元*2）=45元
Thought: 确认客户购买信息
Action:
```
{
  "action": "Human",
  "action_input": "请问您是否确认购买？"
}
```


请问您是否确认购买？
yes

Observation: yes
Thought: 确认客户付款方式
Action:
```
{
  "action": "Human",
  "action_input": "请问您要使用什么付款方式？"
}
```



请问您要使用什么付款方式？
信用卡

Observation: 信用卡
Thought: 获取客户信用卡信息
Action:
```
{
  "action": "Human",
  "action_input": "请提供您的信用卡号和密码，以便完成付款。"
}
```



请提供您的信用卡号和密码，以便完成付款。
12321，dd

Observation: 12321，dd
Thought: 确认客户付款
Action:
```
{
  "action": "purchaseFun",
  "action_input": {
    "creditID": "12321",
    "password": "dd",
    "money": "45"
  }
}
```


Observation: success
Thought: 完成付款
Action:
```
{
  "action": "Final Answer",
  "action_input": "您的付款已完成，感谢您的光临！"
}
```

> Finished chain.


'您的付款已完成，感谢您的光临！'